In [84]:
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.touch_actions import TouchActions
from selenium.webdriver.common.keys import Keys
import pandas as pd
import os
from bs4 import BeautifulSoup as BS
import time
from IPython.display import clear_output

In [106]:
coordinates = ['50:20:0010516', '50:20:0010512', '50:20:0010411']
valid_houses = ['Жилой дом', 'Индивидуальный жилой дом', 'жилой дом', 'Жилое строение', '"Индивидуальный жилой дом"']
houses = []
for coordinate in coordinates:
    coordinate = coordinate.split(':')

    url = 'https://pkk5.rosreestr.ru/#x=4146628.302202864&y=7507512.107243985&z=14&' + \
    'text=%D0%92%20%D0%B3%D1%80%D0%B0%D0%BD%D0%B8%D1%86%D0%B0%D1%85%20%D0%BA%D0%B2%D0%B0%D1%80%D1%82%D0%B0%D0%BB%D0%B0%20'+ \
    str(int(coordinate[0])) + '%3A' + str(int(coordinate[1])) + '%3A' + str(int(coordinate[2])) +'&type=1&app=search&opened=1'

    br = webdriver.Chrome('chromedriver.exe')

    # получаем страницу с объектами
    br.get(url)
    actions = ActionChains(br)

    # нужно выбрать, чтобы поиск был по ОКС
    # щелкаем на выпадающий список
    div = br.find_element_by_id("app-search-form")
    item = div.find_elements_by_class_name("input-group-btn")[0]
    actions.move_to_element(div)
    actions.click(item)
    actions.perform()

    # Ждем, пока откроется, прогрузится
    time.sleep(3)

    # щелкаем на ОКС
    actions = ActionChains(br)
    el = br.find_element_by_class_name('dropdown-menu')
    a = el.find_element_by_id('tag_5')
    actions.click(a)
    actions.perform()

    # Ждем, пока откроется, прогрузится
    time.sleep(5)

    pages = 0

    try:
        # перебираем текущую страницу с домами
        while True:
            elements = br.find_elements_by_class_name('search-result-item')
            len_elements = len(elements)
            for i in range(len_elements):
                
                # нажали на дом
                actions = ActionChains(br)
                element = br.find_elements_by_class_name('search-result-item')[i]
                full = element.find_element_by_tag_name("table")
                actions.click(full)
                actions.perform()

                time.sleep(3)
                
                # разбираем информацию по текущему дому
                info = br.find_element_by_id("feature-oks-info").text.split('\n')
                is_house = False
                is_house_1 = False
                is_house_2 = False
                kad_num = ''
                sq_str = ''
                house_name = ''
                sq_num = 0
                for i in range(1, len(info)):
                    if(info[i-1] == 'Тип:' and info[i] == 'Здание'):
                        is_house_1 = True
                    if(info[i-1] == 'Кад. номер:'):
                        kad_num = info[i]
                    if(info[i-1] == 'Общая площадь:'):
                        sq_str = info[i]
                    if(info[i-1] == 'Наименование:'):
                        house_name = info[i]
                        if(info[i] in valid_houses):
                            is_house_2 = True
                #clear_output(wait = True)
                is_house = is_house_1 and is_house_2
                if(is_house):
                    try:
                        sq_num = float(sq_str[:-len(' кв. м')].replace(',', '.').replace(' ',''))
                    except:
                        sq_num = 0
                    if(sq_num > 1000):
                        houses.append([kad_num, sq_num, house_name])
                        #print('House added. Square = {0} m.'.format(int(sq_num)))
                    #else:
                        #print('House square < 1 000 m')
                #else:
                    #print("It's not house")

                # сворачиваем доп. информацию обратно, чтобы щелкнуть на следующий элемент
                actions = ActionChains(br)
                element = br.find_element_by_class_name('showFullFeature')
                actions.move_to_element(element)
                actions.click(element)
                actions.perform()

                time.sleep(1)
            time.sleep(3)
            pages += 1
            
            # щелкаем на следующую страницу
            next_page = br.find_element_by_link_text('>')
            actions = ActionChains(br)
            actions.move_to_element(next_page)
            actions.click(next_page)
            actions.perform()
            time.sleep(5)
        print('Pages: ', pages)
    except:
        # если что-то случилось во время перехода по страницам по заданному кварталу(чтобы потом глянуть, сколько всего страниц)
        print('Error! Pages: ', pages)
        time.sleep(5)
        continue


Pages:  49
Pages:  7
Pages:  27


In [113]:
# создаем папку для выгрузки csv
output_folder = 'output'
file_name = 'houses.csv'
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

In [115]:
houses = pd.DataFrame(houses)
houses = houses[[0,1]]
houses.to_csv(os.path.join(output_folder, file_name), header=False)

In [94]:
houses[2].value_counts()

Жилой дом                                                                                        27
Индивидуальный жилой дом                                                                         20
-                                                                                                14
индивидуальный жилой дом                                                                          9
жилой дом                                                                                         5
Нежилое здание                                                                                    3
Гараж                                                                                             1
Жилое строение                                                                                    1
"Дом для проживания обучающихся и прихожан» на территории Комплекса зданий Синагоги «Жуковка»     1
Ангар для хранения спецтехники                                                                    1


In [ ]:
html=br.page_source